# RSS: Economista

## 1. Import Libraries

In [76]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image
import urllib
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import numpy as np
from PIL import Image

In [77]:
def get_only_text1(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    return [text,soup.title.text]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

def string_found(string1, string2):
    
    if re.search(r"\b" + re.escape(string1) + r"\b", string2.lower()):
        return True
    return False     
    
def bank_tags(text):
    tags = []
    dict_banks = {'Santander':['santander'],'BBVA':['bbva'],'Bankinter':['bankinter'],'Bankia':['bankia'],
                 'Sabadell':['sabadell'],'ING':['ing'],'Abanca':['abanca'],'Deutsche Bank':['deutsche'],
                  'CaixaBank':['caixabank'],'Openbank':['openbank']}
    for key,values in dict_banks.items():
        for i in values:
            if string_found(i,text.lower()):
                tags.append(key)
                break
    return tags

In [78]:
def economista_news(economista_entries,word_count=100):
    economista = {}
    for i in range(len(economista_entries)):
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,'link':None,
                   'text':None,'current_date':None,'img':None,'premium':None,'tag':None}
        # Periodico
        noticia['newspaper'] = find_between(s=economista_entries[i]['id'], first='www.el', last='.' )
        # Title
        noticia['title'] = economista_entries[i]['title']
        # Headline
        headline =  re.sub(pattern='&quot;',repl='"',string=economista_entries[i]['summary_detail']['value'])
        noticia['headline'] = headline
        # Date
        noticia['date'] = {'date1':economista_entries[i]['published'],'date2':economista_entries[i]['published_parsed']}
        # Link
        noticia['link'] = economista_entries[i]['id']
        # Text
        text = get_only_text(economista_entries[i]['id'])[0][len(headline)+1:]
        if  text.find('\n\t\t\t\tif') != -1:
            index =  text.find('\n\t\t\t\tif')
            text = text[:index]
        noticia['text'] = text
        # Bank Tags
        noticia['tag'] = bank_tags(text)
        # Summarise
        noticia['summarise'] = summarize(text, word_count=word_count)
        # Current Date
        noticia['current_date'] = datetime.datetime.now().timetuple()
        # Image
        noticia['img'] = None
        economista.update({'Noticia'+'_'+str("%02d")%(i+1):noticia})
    return economista

## 2. RSS Economista

In [79]:
url_economista = 'https://www.eleconomista.es/rss/rss-empresas.php'
economista = feedparser.parse(url_economista)
economista_entries = economista['entries']

print('Total Entradas Economista:',len(economista_entries))

Total Entradas Economista: 20


In [72]:
url = economista_entries[0]['id']
print(url)
page=urlopen(url)
soup=BeautifulSoup(page,"html.parser")
text=' '.join(map(lambda p: p.text, soup.find_all('p')))

http://www.eleconomista.es/empresas-finanzas/noticias/10678098/07/20/Transportes-se-reunira-este-miercoles-con-el-camion-para-evitar-la-huelga.html


In [74]:
text.find('\n\t\t\t\tif')

3296

In [75]:
text[:3296]

'El Ministerio de Transportes tiene previsto reunirse este miércoles con el sector del transporte de mercancías por carretera para tratar de acercar posturas y evitar el paro patronal convocado para los próximos 27 y 28 de julio. Según informan fuentes empresariales a este diario, la secretaría general de transportes se puso ayer en contacto con las distintas asociaciones para conocer la disponibilidad de los miembros del grupo negociador del Comité Nacional de Transporte por Carretera, aunque todavía no hay una cita cerrada oficialmente. El Ministerio de Transportes tiene previsto reunirse este miércoles con el sector del transporte de mercancías por carretera para tratar de acercar posturas y evitar el paro patronal convocado para los próximos 27 y 28 de julio. Según informan fuentes empresariales a este diario, la secretaría general de transportes se puso ayer en contacto con las distintas asociaciones para conocer la disponibilidad de los miembros del grupo negociador del Comité Na

In [80]:
economista = economista_news(economista_entries,word_count=100)

In [81]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(economista.keys()):
    if economista[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((economista[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(economista[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(economista[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(economista[i]['headline'])
        print(colored('Summarise','blue'))
        
        if economista[i]['summarise'] != 'Premium Content':
            print(economista[i]['summarise'])
        else:
            print(colored(economista[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(economista[i]['text'])
        display(Markdown('<a href="'+economista[i]['link']+'">Link</a>'))
        print('\n')

ECONOMISTA
Mon, 20 Jul 2020 22:09:04 +0200
La industria del cemento perderá el 20% de su negocio en 2020, la mayor caída en ocho años
Headline
La industria del cemento en España anticipa un deterioro del consumo de cemento en el entorno del 20% al cierre de 2020 como consecuencia del impacto de la crisis por el Covid-19.
Summarise
Con estas previsiones, el consumo de cemento al final del año se situaría por debajo de los 11,5 millones de toneladas, una cifra que lo devuelve al nivel de hace cinco años y que supone casi la tercera parte de la capacidad que tienen las fábricas españolas, lo que amenaza la continuidad de alguna de ellas.
En este entorno, García Brosa, que incide en que "la palabra que mejor describe la situación actual y la previsión para este año es incertidumbre", urge la implantación de "un ajuste en frontera, porque la gran pérdida de exportaciones en el mercado internacional se produce frente a países como Turquía, Egipto, Argelia o Túnez, que no tienen el mismo nive

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10678124/07/20/La-industria-del-cemento-perdera-el-20-de-su-negocio-en-2020-la-mayor-caida-en-ocho-anos.html">Link</a>



ECONOMISTA
Mon, 20 Jul 2020 21:41:36 +0200
Transportes se reunirá este miércoles con el camión para evitar la huelga
Headline
El Ministerio de Transportes tiene previsto reunirse este miércoles con el sector del transporte de mercancías por carretera para tratar de acercar posturas y evitar el paro patronal convocado para los próximos 27 y 28 de julio. Según informan fuentes empresariales a este diario, la secretaría general de transportes se puso ayer en contacto con las distintas asociaciones para conocer la disponibilidad de los miembros del grupo negociador del Comité Nacional de Transporte por Carretera, aunque todavía no hay una cita cerrada oficialmente.
Summarise
Según informan fuentes empresariales a este diario, la secretaría general de transportes se puso ayer en contacto con las distintas asociaciones para conocer la disponibilidad de los miembros del grupo negociador del Comité Nacional de Transporte por Carretera, aunque todavía no hay una cita cerrada oficialmente.
"La

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10678098/07/20/Transportes-se-reunira-este-miercoles-con-el-camion-para-evitar-la-huelga.html">Link</a>



ECONOMISTA
Mon, 20 Jul 2020 21:23:26 +0200
Norges Bank aumenta su participación en Sabadell por encima del 3%
Headline
Norges Bank ha incrementado su participación en Banco Sabadell hasta superar el 3% de su capital social, según consta en los registros de la Comisión Nacional del Mercado de Valores (CNMV).
Summarise
Norges Bank ha incrementado su participación en Banco Sabadell hasta superar el 3% de su capital social, según consta en los registros de la Comisión Nacional del Mercado de Valores (CNMV).
Esta operación supone, asimismo, la primera transacción que Norges Bank lleva a cabo desde enero de 2019 en Banco Sabadell, cuando remitió su última actualización al regulador, en la que reducía por primera vez su participación por debajo del 3%.
De esta forma, la entidad noruega vuelve a posicionarse como uno de los únicos tres accionistas mayoritarios de Sabadell con una participación por encima de este umbral del 3%, solo por detrás de Sanders Capital y Fintech Europe.
Text
Norges 

...

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10677046/07/20/El-numero-de-gasolineras-bajo-en-2019-tras-una-decada-de-constantes-subidas.html">Link</a>



ECONOMISTA
Mon, 20 Jul 2020 13:33:54 +0200
Renfe creará una cola virtual para evitar que la web colapse con las ofertas
Headline
Renfe da un paso más en la estrategia de mejora de su página web, una de las más criticadas del mercado. El operador ferroviario ya ha iniciado la transformación de su portal con "el estreno de un nuevo diseño y mejoras en la navegabilidad y usabilidad" y ahora tiene previsto activar un sistema de colas virtuales para evitar colapsos y abusos durante las grandes ofertas, como los 1.000 billetes de Avlo al día por cinco euros o los 250.000 billetes a 25 euros para celebrar el 25 aniversario del AVE. Con ambas promociones la web de Renfe se bloqueó
Summarise
Los pliegos del contrato, valorado en 216.000 euros, especifican que se deben crear varios tipos de colas y estadios de acceso para evitar abusos en las promociones y que tengan ventaja los usuarios que han accedido a la web antes de que arranque la promoción.
Por ejemplo, en el contexto de internacionali

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10676875/07/20/Renfe-creara-una-cola-virtual-para-evitar-que-la-web-colapse-con-las-ofertas-.html">Link</a>



ECONOMISTA
Mon, 20 Jul 2020 13:21:33 +0200
Los incendios forestales bajan un 40% este 2020 por el confinamiento y el clima
Headline
Más lluvias en mayo y junio, y el confinamiento para frenar el estallido de la Covid-19 -por las restricciones a las actividades agrícolas y ganaderas-, han reducido un 40% el número de incendios forestales en España en lo que va de 2020 y en un 55% la superficie arrasada. Para el resto del año, las autoridades se mantienen "nada tranquilas", aunque señalan que "vamos bien".
Summarise
En su opinión, el descenso se debe a la abundancia de lluvias durante esta primavera, a unas temperaturas suaves y a la crisis de la Covid-19: "el confinamiento ha ayudado a que hubiera pocos incendios, porque el ser humano es la causa fundamental, y al no estar el ser humano en el monte" ha apuntado, antes de añadir que "pero no porque no hayamos ido de parrillada, sino porque muchas de las actividades rurales que generan incendios forestales de una manera voluntaria o in

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10676828/07/20/Los-incendios-forestales-bajan-un-40-este-2020-por-el-confinamiento-y-el-clima.html">Link</a>